In [1]:
%load_ext Cython
# include Cython to accelerate the code


# Output the transformed data (OD_region&period.csv) 

# Output the index (index_50*50.csv)

In [2]:
%%cython

import pandas as pd
import numpy as np
import time
import math
import csv
import pyproj

WebMercator = pyproj.Proj(init='epsg:3857') #定義WebMercator坐標系
WGS84 = pyproj.Proj(init='epsg:4326') #定義WGS84坐標系



#set parameters
##############
cdef int n=50   
cdef int m=50
cdef int start_time=0
cdef int end_time=24
cdef int time_zone=48

cdef double dx=300  #margin
cdef double dy=200

###############



cdef double P=(end_time-start_time)/time_zone*60

start = time.time()

#read origin OD data
data_A = pd.read_csv('dockless-vehicles-7.csv');
numpy_array_str = np.genfromtxt("dockless-vehicles-7.csv", delimiter=",", dtype=str,skip_header=1)


numpy_array = np.genfromtxt("dockless-vehicles-7.csv", delimiter=",", skip_header=1)
numpy_array=np.c_[ numpy_array, np.ones(len(numpy_array)), np.ones(len(numpy_array)), np.ones(len(numpy_array)), np.ones(len(numpy_array)) ] 
for g in range(0,len(numpy_array)):
    numpy_array[g][14],numpy_array[g][13]=pyproj.transform(WGS84, WebMercator,numpy_array[g][8],numpy_array[g][7])#始經,緯 x,y
    numpy_array[g][16],numpy_array[g][15]=pyproj.transform(WGS84, WebMercator,numpy_array[g][10],numpy_array[g][9])#終經,緯 x,y
    


    
    
cdef double x_max=max(data_A['StartLongitude'].max(),data_A['EndLongitude'].max())
cdef double x_min=min(data_A['StartLongitude'].min(),data_A['EndLongitude'].min())
cdef double y_max=max(data_A['StartLatitude'].max(),data_A['EndLatitude'].max())
cdef double y_min=min(data_A['StartLatitude'].min(),data_A['EndLatitude'].min())

cdef (double, double) a=pyproj.transform(WGS84, WebMercator,x_min,y_max)
cdef (double, double) b=pyproj.transform(WGS84, WebMercator,x_max,y_max)
cdef (double, double) c=pyproj.transform(WGS84, WebMercator,x_min,y_min)
cdef (double, double) d=pyproj.transform(WGS84, WebMercator,x_max,y_min)#經緯xy




cdef (double, double) pos_1=(a[1] + dy/2,   a[0] - dx/2)  #緯,經 左上
cdef (double, double) pos_mn=(d[1] - dy/2,   d[0] + dx/2)  #緯,經 右下


dx=abs(pos_1[1]-pos_mn[1])/m
dy=abs(pos_1[0]-pos_mn[0])/n

#Output index
with open('index_50*50.csv', 'w', newline='') as csvfile:

    writer = csv.writer(csvfile)

    writer.writerow(['m=',m])
    writer.writerow(['n=',n])
    writer.writerow(['dx=',dx,'m'])
    writer.writerow(['dy=',dy,'m'])
    writer.writerow(['start_time=',start_time])
    writer.writerow(['end_time=',end_time])
    writer.writerow(['time_zone=',time_zone])
    #writer.writerow(['upper left=',pos_1])
    #writer.writerow(['lower right=',pos_mn])

    writer.writerow(['k', "i", "j","center_x_WebMercator","center_y_WebMercator","center_x_WGS84","center_y_WGS84"])
    for ii in range(1,m+1):
        for jj in range(1,n+1):
            writer.writerow([(ii-1)*n+jj, ii, jj, a[0]+dx*(ii-1) , a[1]-dy*(jj-1),   pyproj.transform(WebMercator, WGS84, a[0]+dx*(ii-1) ,a[1]-dy*(jj-1))[0], pyproj.transform(WebMercator, WGS84, a[0]+dx*(ii-1) ,a[1]-dy*(jj-1))[1]    ])

            
#####################
#region&period partition

#filename='OD_num_' + m +'*'+n+'.csv'
    
cdef double Lat,Lon,period1,period2


cdef int i,j,k1,k2




#the function that calculate the point number 
def sumup(int i, int j, int n):
    cdef int sum
    sum = (i-1)*n+j
    return sum


print("dx = ",dx)
print("dy = ",dy)
print("pos_1 = ",pos_1)

####output region num
with open('OD_region&period.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)

  # write the header to the output
    writer.writerow(['TripID', 'StartCell', 'EndCell','StartDate','StartPeriod','EndDate','EndPeriod'])

    for g in range(0,len(data_A)):

        #print("g=",g)
        
        #start cell
        Lat=numpy_array[g][13]
        Lon=numpy_array[g][14]
        j= abs( int((Lat-pos_1[0])/dy)   )+1 #緯
        i= abs( int((Lon-pos_1[1])/dx)   )+1 #經
        #print("(",i,",",j,")")
        k1=sumup(i,j,n)
        #print("k1 = ",k1)
        #print("(",i,",",j,")")
        
        
        
        #end cell
        Lat=numpy_array[g][15]
        Lon=numpy_array[g][16]
        j= abs( int((Lat-pos_1[0])/dy)   )+1
        i= abs( int((Lon-pos_1[1])/dx)   )+1
        #print("(",i,",",j,")")
        k2=sumup(i,j,n)

        #print("(",i,",",j,")")
        #print("k2 = ",k2)
    
    
    
###output period partition
        #stard period
        if len(numpy_array_str[g][2])==5:
            period1 = int((int(numpy_array_str[g][2][0:2]) * 60+int(numpy_array_str[g][2][3:5] ))/P)+1
            #print(period)
        
        elif len(numpy_array_str[g][2])==4:
            period1 = int((int(numpy_array_str[g][2][0:1]) * 60+int(numpy_array_str[g][2][2:4] ))/P)+1
            #print(period)
        else:
            period1 = 24*60/P
            #print(period)

            
        #end period    
        if len(numpy_array_str[g][4])==5:
            period2 = int((int(numpy_array_str[g][4][0:2]) * 60+int(numpy_array_str[g][4][3:5] ))/P)+1
            #print(period)
        
        elif len(numpy_array_str[g][4])==4:
            period2 = int((int(numpy_array_str[g][4][0:1]) * 60+int(numpy_array_str[g][4][2:4] ))/P)+1
            #print(period)
        else:
            period2 = 24*60/P
        
        # write startPoint and endPoint to the output
        writer.writerow([data_A['TripID'][g], k1, k2,data_A['StartDate'][g],period1,data_A['EndDate'][g],period2])
        
    

        
        
        
        
        

end = time.time()

cy_time = end - start

print("Cython time = {}".format(cy_time))






dx =  504.7113187538087
dy =  310.1360176192224
pos_1 =  (4620999.974214424, -9556371.68714857)
Cython time = 48.974205017089844
